In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [2]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []
keywords = 'ganjar'

In [3]:
def scrape_links(page_number,keywords):
    url=f"https://www.beritajakarta.id/news_index/search?q={keywords}&p={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "media blog-list news-index"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [4]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [5]:
def scrape_url(url,keywords):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1')
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
                
            elem = soup.find('div', {"class": "text-secondary"})
            
            if elem:
                input_string = elem.text
                parts = input_string.split("Reporter :")
                date_text = parts[0].strip()
                editor_and_view = parts[1].split("Editor :")
                author_text = editor_and_view[0].strip()
                remaining_text = editor_and_view[1]
                editor_parts = remaining_text.split()
                editor_text = editor_parts[0]
                viewer_text = editor_parts[-1]
            else:
                author_text = "Author not found"
                date_text = "Date not found"
                editor_text = "Editor not found"
                viewer_text = "Viewer not found"
                
                
            # Author berita
            # author_elem = soup.find('div', {"class": "jeg_meta_author"})
            # if author_elem:
            #     author_text = author_elem.find('a')
            #     author_text = author_text.text
            # else:
            #     author_text = "Author not found"
                     
            # tanggal berita
            # date_elem = soup.find('div', {"class": "jeg_meta_date"})
            # if date_elem:
            #     date_text = date_elem.find('a')
            #     date_text = date_text.text
            # else:
            #     date_text = "Date not found"
            # Category berita
            category_elements = soup.find_all('li', {"class":"breadcrumb-item"}) 
            if category_elements:
                category_text= category_elements[1].text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "news-content"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'keywords':keywords,
                            'author' : author_text,
                            'category':category_text,
                            'editor' : editor_text,
                            'view' : viewer_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jakarta',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [6]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [7]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  10


,title,keywords,author,category,editor,view,date,content,region,link
0,Gubernur Jateng Kunjungi Balai Kota DKI,ganjar,Erna Martiyanti,Jakarta Hari Ini,Nani,3802,"Kamis, 25 Februari 2016 13:19","Gubernur DKI Jakarta, Basuki Tjahaja Purnama m...",jakarta,https://www.beritajakarta.id/read/26306/gubern...
1,Basuki Setujui Istri Ganjar Pindah Tugas,ganjar,Erna Martiyanti,Pemerintahan,Nani,3393,"Kamis, 25 Februari 2016 16:41","Gubernur DKI Jakarta, Basuki Tjahaja Purnama m...",jakarta,https://www.beritajakarta.id/read/26327/basuki...
2,Pelayanan Publik Prima di DKI Diganjar Penghar...,ganjar,Aldi Geri Lumban Tobing,Fokus,Toni,3331,"Jumat, 12 Maret 2021 16:57",Pemerintah Provinsi DKI Jakarta melalui Dinas ...,jakarta,https://www.beritajakarta.id/read/87643/pelaya...
3,JXB Diganjar Tiga Bintang di TOP BUMD Awards 2021,ganjar,Aldi Geri Lumban Tobing,BUMD,Toni,1468,"Jumat, 10 September 2021 21:39",Jakarta Experience Board (JXB)/PT Jakarta Tour...,jakarta,https://www.beritajakarta.id/read/92075/jxb-di...
4,Djarot akan Beradu Akting dengan Ganjar Pranowo,ganjar,Suriaman Panjaitan,Pendidikan,Nani,3700,"Minggu, 14 Agustus 2016 17:34","Wakil Gubernur DKI Jakarta, Djarot Saiful Hida...",jakarta,https://www.beritajakarta.id/read/35070/djarot...
5,21 Wisatawan Ancol Diganjar Sanksi Sosial,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,1053,"Jumat, 30 Oktober 2020 16:09",21 wisatawan yang mengunjungi kawasan Taman I...,jakarta,https://www.beritajakarta.id/read/84315/21-wis...
6,Anies Ajak Ganjar dan Emil Sambangi Balai Kota...,ganjar,Mustaqim Amna,Pemerintahan,Toni,2819,"Senin, 05 November 2018 19:45","Gubernur DKI Jakarta, Anies Baswedan mengajak ...",jakarta,https://www.beritajakarta.id/read/63432/anies-...
7,Inovasi Vaksinasi Remaja Puskesmas Kecamatan P...,ganjar,Suparni,Kesehatan,Toni,2122,"Selasa, 30 November 2021 10:20","Puskesmas Kecamatan Penjaringan, Jakarta Utara...",jakarta,https://www.beritajakarta.id/read/93717/inovas...
8,"Cepat Respons Aduan Masyarakat, Kelurahan Kela...",ganjar,Wuri Setyaningsih,Pemerintahan,Erikyanri,1279,"Senin, 12 April 2021 17:42","Kelurahan Kelapa Dua, Kecamatan Kebon Jeruk, J...",jakarta,https://www.beritajakarta.id/read/88417/cepat-...
9,Kejujuran Karyawan JXB Ini Diganjar Hadiah Men...,ganjar,Aldi Geri Lumban Tobing,BUMD,Andry,735,"Jumat, 18 Februari 2022 08:08","Muhammad Iqbal, salah seorang karyawan Jakarta...",jakarta,https://www.beritajakarta.id/read/95331/kejuju...


In [9]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 10


,title,keywords,author,category,editor,view,date,content,region,link
0,Gubernur Jateng Kunjungi Balai Kota DKI,ganjar,Erna Martiyanti,Jakarta Hari Ini,Nani,3802,"Kamis, 25 Februari 2016 13:19","Gubernur DKI Jakarta, Basuki Tjahaja Purnama m...",jakarta,https://www.beritajakarta.id/read/26306/gubern...
1,Basuki Setujui Istri Ganjar Pindah Tugas,ganjar,Erna Martiyanti,Pemerintahan,Nani,3393,"Kamis, 25 Februari 2016 16:41","Gubernur DKI Jakarta, Basuki Tjahaja Purnama m...",jakarta,https://www.beritajakarta.id/read/26327/basuki...
2,Pelayanan Publik Prima di DKI Diganjar Penghar...,ganjar,Aldi Geri Lumban Tobing,Fokus,Toni,3331,"Jumat, 12 Maret 2021 16:57",Pemerintah Provinsi DKI Jakarta melalui Dinas ...,jakarta,https://www.beritajakarta.id/read/87643/pelaya...
3,JXB Diganjar Tiga Bintang di TOP BUMD Awards 2021,ganjar,Aldi Geri Lumban Tobing,BUMD,Toni,1468,"Jumat, 10 September 2021 21:39",Jakarta Experience Board (JXB)/PT Jakarta Tour...,jakarta,https://www.beritajakarta.id/read/92075/jxb-di...
4,Djarot akan Beradu Akting dengan Ganjar Pranowo,ganjar,Suriaman Panjaitan,Pendidikan,Nani,3700,"Minggu, 14 Agustus 2016 17:34","Wakil Gubernur DKI Jakarta, Djarot Saiful Hida...",jakarta,https://www.beritajakarta.id/read/35070/djarot...
5,21 Wisatawan Ancol Diganjar Sanksi Sosial,ganjar,Budhi Firmansyah Surapati,Lintas Kota,Andry,1053,"Jumat, 30 Oktober 2020 16:09",21 wisatawan yang mengunjungi kawasan Taman I...,jakarta,https://www.beritajakarta.id/read/84315/21-wis...
6,Anies Ajak Ganjar dan Emil Sambangi Balai Kota...,ganjar,Mustaqim Amna,Pemerintahan,Toni,2819,"Senin, 05 November 2018 19:45","Gubernur DKI Jakarta, Anies Baswedan mengajak ...",jakarta,https://www.beritajakarta.id/read/63432/anies-...
7,Inovasi Vaksinasi Remaja Puskesmas Kecamatan P...,ganjar,Suparni,Kesehatan,Toni,2122,"Selasa, 30 November 2021 10:20","Puskesmas Kecamatan Penjaringan, Jakarta Utara...",jakarta,https://www.beritajakarta.id/read/93717/inovas...
8,"Cepat Respons Aduan Masyarakat, Kelurahan Kela...",ganjar,Wuri Setyaningsih,Pemerintahan,Erikyanri,1279,"Senin, 12 April 2021 17:42","Kelurahan Kelapa Dua, Kecamatan Kebon Jeruk, J...",jakarta,https://www.beritajakarta.id/read/88417/cepat-...
9,Kejujuran Karyawan JXB Ini Diganjar Hadiah Men...,ganjar,Aldi Geri Lumban Tobing,BUMD,Andry,735,"Jumat, 18 Februari 2022 08:08","Muhammad Iqbal, salah seorang karyawan Jakarta...",jakarta,https://www.beritajakarta.id/read/95331/kejuju...


save data

In [10]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/beritaJakarta_{current_datetime}.xlsx'
# filtered_df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')